<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/spellcheck_I_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5_epoch50%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 9.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595716 sha256=18a850d703e96ba6c0a95e6395318d938b1efdf7a44bfaba2248120fc625f94a
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 8.1 MB/s 
     |████████████████████████████████| 769 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 87.9 MB/s 
     |████████████████████████████████| 3.0 MB 83.4 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-mdng68zw
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_G_inc.csv		   df_P_inc.csv
'Colab Notebooks'	 df_H_inc_노이.csv	  'My Drive'
 C_ori_model.pt		 df_J_inc_again_노이.csv   Q_inc_model.pt
'답안 작성용 파일.csv'	 df_J_inc.csv		   spell_check
 df_C_inc.csv		 df_J_inc_dain.csv
'# df_G_inc.csv'	 df_K_inc.csv


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/spell_check/I_spell_check.csv')

## 데이터 전처리

In [ ]:
df['digit_3'].value_counts()

561    123703
562     49756
551     12549
559      1417
Name: digit_3, dtype: int64

In [ ]:
df.loc[(df['digit_3'] == 551), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 559), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 561), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 562), 'digit_3'] = 3  

In [ ]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(int(label))

    data_list.append(data)

print(data_list[:5])

[['음식점에서 접객시설을 갖추고 참치 회(일본식)', 2], ['음식점 접객시설을 갖추고 냉면', 2], ['음식점에서 접객시설을 갖추고 회', 2], ['유흥주점에서 무도 시설을 갖추고 술판매', 3], ['음료점에서 빙수, 아이스크림, 커피, 디저트 제공 커피. 빙수 등을 제공', 2]]


## Train data & test data

In [ ]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 4086, 5370, 6712, 7088,  827, 5439, 2861, 6897,  517, 5796,
        6314, 6642,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(14, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2636 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.4524668455123901 train acc 0.171875
epoch 1 batch id 201 loss 1.0829689502716064 train acc 0.3655939054726368
epoch 1 batch id 401 loss 0.5762678980827332 train acc 0.5225218204488778
epoch 1 batch id 601 loss 0.2958528399467468 train acc 0.6537281613976705
epoch 1 batch id 801 loss 0.07955168187618256 train acc 0.7332045880149812
epoch 1 batch id 1001 loss 0.17813505232334137 train acc 0.7811563436563437
epoch 1 batch id 1201 loss 0.10401736199855804 train acc 0.8138270191507078
epoch 1 batch id 1401 loss 0.03198309615254402 train acc 0.8378501962883654
epoch 1 batch id 1601 loss 0.18007081747055054 train acc 0.8558615708931917
epoch 1 batch id 1801 loss 0.33114004135131836 train acc 0.8702196696279845
epoch 1 batch id 2001 loss 0.11663161218166351 train acc 0.8816216891554223
epoch 1 batch id 2201 loss 0.01680515706539154 train acc 0.8909799522944116
epoch 1 batch id 2401 loss 0.03862879052758217 train acc 0.8989093086214077
epoch 1 batch id 2601 loss 0.0212

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 3 batch id 1 loss 0.012816069647669792 train acc 1.0
epoch 3 batch id 201 loss 0.0459149032831192 train acc 0.9856965174129353
epoch 3 batch id 401 loss 0.008545688353478909 train acc 0.9860504987531172
epoch 3 batch id 601 loss 0.028337109833955765 train acc 0.9860128951747088
epoch 3 batch id 801 loss 0.011673451401293278 train acc 0.986286672908864
epoch 3 batch id 1001 loss 0.16963109374046326 train acc 0.9862637362637363
epoch 3 batch id 1201 loss 0.0739804357290268 train acc 0.9861703788509575
epoch 3 batch id 1401 loss 0.003859417513012886 train acc 0.9860925231977159
epoch 3 batch id 1601 loss 0.05997567996382713 train acc 0.985917004996877
epoch 3 batch id 1801 loss 0.23184792697429657 train acc 0.9861361743475847
epoch 3 batch id 2001 loss 0.07137779146432877 train acc 0.9863818090954523
epoch 3 batch id 2201 loss 0.004426805302500725 train acc 0.9862704452521581
epoch 3 batch id 2401 loss 0.02069712243974209 train acc 0.9864900041649313
epoch 3 batch id 2601 loss 0.015

  0%|          | 0/293 [00:00<?, ?it/s]

epoch 3 test acc 0.9822331678560348


  0%|          | 0/2636 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.016385585069656372 train acc 1.0
epoch 4 batch id 201 loss 0.031189069151878357 train acc 0.9849191542288557


In [ ]:
#학습 모델 저장
torch.save(model, 'I_spell_model_epoch50.pt')